In [ ]:
from lmptools import Dump, DumpCallback, DumpSnapshot
from lmptools.exceptions import SkipSnapshot
from lmptools.polymers import Polymer
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import os

In [ ]:
sqlite_dr_vs_rg_db = "dr_vs_rg.db"
num_rods = 125
num_polymers = 1000
tselect = np.random.choice(np.arange(13000000, 14002000, 2000), 25)
polymer_type = 1
rod_type = 2

rg0 = 2.7
# histogram bins
rg_bins = np.linspace(1.0, 2.5*2.7, 30)/rg0

dr_cutoff = 5.0 # 2*rc

# Callbacks
class Skipsnapshots(DumpCallback):
    def __init__(self, timestamps):
        self.timestamps = timestamps

    def on_snapshot_parse_timestamp(self, timestamp: int, *args, **kwargs):
        if timestamp not in self.timestamps:
            raise SkipSnapshot(f"Snapshot for timestamp {timestamp} skipped")

class RadiusOfGyrationAnalysis(DumpCallback):
    def __init__(self, sqlengine):
        self.sqlengine = sqlengine

    def on_snapshot_parse_end(self, snapshot: DumpSnapshot, *args, **kwargs):
        sbox = snapshot.box
        atoms_df = snapshot.dataframe

        # Filter polymer atoms from the snapshot
        rod_atoms = [atom for atom in snapshot.atoms if atom.type == rod_type]
        assert len(rod_atoms) == 8000

        polymer_atoms = [atom for atom in snapshot.atoms if atom.type == polymer_type]
        assert len(polymer_atoms) == 32000

        rigid_df = atoms_df[atoms_df['type'] == rod_type]
        polymer_df = atoms_df[atoms_df['type'] == polymer_type]

        rigid_df_temp = pd.DataFrame(columns=["xcm", "ycm", "zcm"])
        rigid_df_temp['xcm'] = np.mean(np.split(np.mean(np.split(rigid_df['xu'], num_rods*16), axis=1), num_rods), axis=1)
        rigid_df_temp['ycm'] = np.mean(np.split(np.mean(np.split(rigid_df['yu'], num_rods*16), axis=1), num_rods), axis=1)
        rigid_df_temp['zcm'] = np.mean(np.split(np.mean(np.split(rigid_df['zu'], num_rods*16), axis=1), num_rods), axis=1)

        polymer_df_temp = pd.DataFrame(columns=["xcm", "ycm", "zcm", "rg"])
        polymer_df_temp['xcm'] = np.mean(np.split(polymer_df['xu'], 1000), axis=1)
        polymer_df_temp['ycm'] = np.mean(np.split(polymer_df['yu'], 1000), axis=1)
        polymer_df_temp['zcm'] = np.mean(np.split(polymer_df['zu'], 1000), axis=1)

        # Compute the Radius of gyration of all polymers in this snapshot
        polymer_rg = []
        for polymeratoms in [polymer_atoms[i*32:(i*32)+32] for i in np.arange(1000)]:
            polymer_rg.append(Polymer(atoms=polymeratoms).rg)
        polymer_df_temp["rg"] = polymer_rg

        res = {}
        res["dr"] = []
        res["rg"] = []

        for rigidcm in rigid_df_temp.values:
            for chaincm in polymer_df_temp.values:
                dx = rigidcm[0] - chaincm[0]
                nx = np.floor(dx/sbox.Lx)
                dx = dx - nx*sbox.Lx

                dy = rigidcm[1] - chaincm[1]
                ny = np.floor(dy/sbox.Ly)
                dy = dy - ny*sbox.Ly

                dz = rigidcm[2] - chaincm[2]
                nz = np.floor(dz/sbox.Lz)
                dz = dz - nz*sbox.Lz

                res["dr"].append(np.sqrt(dx**2 + dy**2 + dz**2))
                res["rg"].append(chaincm[3])

        # convert res to dataframe and write to db
        df = pd.DataFrame(res)
        df.to_sql("dr_vs_rg", con=self.sqlengine, if_exists='append')
        print(f"Finished processing snapshot {snapshot.timestamp}")
        return

In [ ]:
for root, dirs, files in os.walk("/storage/backups/seagate/polymer_melts/simulations/larger_system/phi0.2_r125_p1000/"):
    for d in dirs:
        if d in ['run1', 'run2', 'run3', 'run4', 'run5']:
            rigid_dump_file = os.path.join(os.path.join(root, d), "dump.rigidChains.lammpstrj")
            polymer_dump_file = os.path.join(os.path.join(root, d), "dump.flexChains.lammpstrj")
            merged_dump_file = os.path.join(os.path.join(root, d), "dump.melt.lammpstrj")

            drigid = Dump(rigid_dump_file, unwrap=True, callback=Skipsnapshots(tselect))
            dpolymer = Dump(polymer_dump_file, unwrap=True, callback=Skipsnapshots(tselect))

            # merge the dump files into one
            with open(merged_dump_file, 'w') as f:
                for rigid_snap, polymer_snap in zip(drigid, dpolymer):
                    f.write(str(rigid_snap + polymer_snap)+"\n")
            f.close()

            # Use the merged dumpfile to do the rg analysis
            sql_db = os.path.join(os.path.join(root, d), sqlite_dr_vs_rg_db)
            engine = create_engine(f"sqlite:///{sql_db}", echo=False)
            dmelt = Dump(merged_dump_file, unwrap=True, callback=RadiusOfGyrationAnalysis(engine))
            dmelt.parse(persist=False)
            print(f"Finished {d}")